# Power Transformation - box-cox and yeo-johnson

In [116]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score

from scipy.stats import stats
import seaborn as sns
import plotly as plt
import matplotlib as mlt


In [117]:
df = pd.read_csv("../concrete_data.csv")

In [118]:
df.sample(10)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
650,122.6,183.9,0.0,203.5,0.0,958.2,800.1,90,33.19
995,158.6,148.9,116.0,175.1,15.0,953.3,719.7,28,27.68
30,304.0,76.0,0.0,228.0,0.0,932.0,670.0,365,55.26
570,295.8,0.0,0.0,185.7,0.0,1091.4,769.3,28,25.22
437,173.5,50.1,173.5,164.8,6.5,1006.2,793.5,28,38.20
889,262.0,111.0,86.0,195.0,5.0,895.0,733.0,28,33.72
39,237.5,237.5,0.0,228.0,0.0,932.0,594.0,180,36.25
71,313.3,262.2,0.0,175.5,8.6,1046.9,611.8,3,28.80
251,250.0,0.0,95.7,187.4,5.5,956.9,861.2,28,29.22
514,424.0,22.0,132.0,168.0,8.9,822.0,750.0,56,74.36


In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              1030 non-null   float64
 1   Blast Furnace Slag  1030 non-null   float64
 2   Fly Ash             1030 non-null   float64
 3   Water               1030 non-null   float64
 4   Superplasticizer    1030 non-null   float64
 5   Coarse Aggregate    1030 non-null   float64
 6   Fine Aggregate      1030 non-null   float64
 7   Age                 1030 non-null   int64  
 8   Strength            1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


In [120]:
df.describe()


,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [121]:
df.iloc[:, 0:8]

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360
...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28


In [122]:
X = df.drop(columns=["Strength"])
Y = df.iloc[:, -1]
# X+=0.000001


In [123]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3, random_state=42)


In [124]:
trf1 = ColumnTransformer([("Scaler", StandardScaler(with_mean=True, with_std=True), [0,1,2,3,4,5,6,7])])

In [125]:
trf2 = PowerTransformer(method="yeo-johnson")

In [126]:
trf3 = LinearRegression()

In [127]:
pipe = Pipeline([("Scaler",trf1), ("Power-Transformer", trf2), ("model", trf3)])

In [128]:
pipe.fit(Xtrain, Ytrain)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('Scaler', ...), ('Power-Transformer', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('Scaler', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers co

In [129]:
pred1 = pipe.predict(Xtest)
pred1

array([49.41064757, 45.37081002, 63.88810463, 42.66092057, 12.54130452,
       45.79178272, 28.25582712, 52.49124812, 30.59392505, 46.71716205,
       31.8500052 , 15.23934859, 54.23268228, 41.23550697, 25.22095844,
       29.10290262, 31.12625812, 22.80241473, 39.01432668, 28.47405417,
       41.09650266, 35.62449969, 45.66133933, 12.35106078, 32.53350608,
       35.60256587,  8.94822209, 46.93578529, 46.640586  , 14.08663193,
       39.06746541, 33.42917183, 46.12304907, 53.34761848, 21.6967632 ,
       36.38773551, 30.30030337, 45.18572891, 10.26318002, 46.19556415,
       16.57889276,  3.79367221, 41.43307098, 52.38675966, 11.20571583,
       64.72761213, 49.74551493, 47.29830686, 23.67303148,  7.3349305 ,
       44.83615051, 47.26391266, 27.33996581, 21.66351677, 46.50914097,
       42.27630187, 23.63077075, 11.31761359, 35.60292584, 22.52044867,
       45.03263279, 13.34163601, 39.36581904, 49.49041646, 27.99089883,
       20.6738563 , 38.27444034, 16.77438809, 32.59211195, 22.56

In [130]:
r2_score(Ytest, pred1)

0.8113091603611311

In [131]:
np.mean(cross_val_score(pipe, X, Y, scoring="r2" ))

0.7288496619106999